In [76]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import math
from collections import defaultdict



In [11]:
def display_graph(graph_edges):
    
    G = nx.Graph()
    
    G.add_edges_from(graph_edges)
    
    nx.draw(G, with_labels=True, font_weight='bold')
    
    plt.title('Undirected Unweighted Graph')
    plt.show()

Part 2 #4

In [27]:
def get_number_of_vertices(graph_edges):
    return len(set(np.array(graph_edges).flatten()))

Part 2 #5

In [46]:
def get_unique_graph(graph_edges):
    for i in range(len(graph_edges)):
        edge = graph_edges[i]
        
        if edge[0] > edge[1]:
            graph_edges[i] = (edge[1], edge[0])
    
    
    unique_edges = set(graph_edges)
    
    return unique_edges

def get_vertex_degree(graph_edges, vertex):
    unique_edges = get_unique_graph(graph_edges)
    
    vertex_degree = 0
    for edge in unique_edges:
        if vertex in edge:
            vertex_degree += 1
    
    return vertex_degree
    

Part 2 #6

In [71]:
def calculate_clustering_coefficient(number_of_neighbors, actual_connections):
    if number_of_neighbors == 1:
        return 0
    possible_neighbors = math.comb(number_of_neighbors, 2)
    return actual_connections / possible_neighbors

def get_connections(graph_edges, vertex):
    edges = []
    for edge in graph_edges:
        if vertex in edge:
            edges.append(edge)
            
    return set(np.array(edges).flatten())

def get_subgraph(graph_edges, vertex):
    unique_edges = get_unique_graph(graph_edges)
    
    connections = get_connections(unique_edges, vertex)
        
    subgraph = []
    
    for edge in unique_edges:
        if all(num in connections for num in edge):
            subgraph.append(edge)
            
    return subgraph

def get_vertex_clustering_coefficient(graph_edges, vertex):
    vertex_degree = get_vertex_degree(graph_edges, vertex)
    
    subgraph = get_subgraph(graph_edges, vertex)
    
    actual_connections = len([pair for pair in subgraph if vertex not in pair])
    
    return calculate_clustering_coefficient(vertex_degree, actual_connections)
    

Part 2 #7

In [114]:
def get_betweenness_centrality(graph_edges, vertex):
    G = nx.Graph()
    G.add_edges_from(graph_edges)

    shortest_paths = dict(nx.all_pairs_shortest_path(G))

    target_betweenness_centrality = 0

    for source, paths in shortest_paths.items():
        for end, path in paths.items():
            if vertex in path[1:-1]:
                total_shortest_paths = len(shortest_paths[source][end])
                target_betweenness_centrality += path.count(vertex) / total_shortest_paths

    return target_betweenness_centrality

Part 2 #8

In [93]:
def get_average_shortest_path_length(graph_edges):
    G = nx.Graph()
    G.add_edges_from(graph_edges)
    
    shortest_path_lengths = []
    nodes = G.nodes()
    for node1 in nodes:
        for node2 in nodes:
            if node1 != node2:
                shortest_path_lengths.append(nx.shortest_path_length(G, source=node1, target=node2))

    average_shortest_path_length = sum(shortest_path_lengths) / len(shortest_path_lengths)
    return average_shortest_path_length
    
    

Part 2 #9

In [98]:
# NOTE this function assumes a 0-indexed graph and treats all as such

def get_adjacency_matrix(graph_edges):
    num_nodes = max(max(edge) for edge in graph_edges) + 1
    
    adjacency_matrix = np.zeros((num_nodes, num_nodes))
    
    for edge in graph_edges:
        node1, node2 = edge
        adjacency_matrix[node1][node2] = 1
        adjacency_matrix[node2][node1] = 1  
    
    return adjacency_matrix

In [115]:
def main():
    input_graph = [
        (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1,10),
        (2, 7), (2,8), (2,9),
        (3, 4), (3, 6), (3, 8),
        (4, 8),
        (5, 7),
        (6, 8),
        (7, 8), (7, 9),
        (8, 9), (8, 10),
        (9, 10),
        (10, 11),
        (2, 1),
        (10, 11),
        (7, 1)
    ] 
    
    G = nx.Graph()
    G.add_edges_from(input_graph)
    
    print(get_number_of_vertices(input_graph)) # 11
    print(get_vertex_degree(input_graph, 1)) # 6
    print(get_vertex_clustering_coefficient(input_graph, 1)) # 0.190
    print(get_betweenness_centrality(input_graph, 1)) # 0.293
    print(get_average_shortest_path_length(input_graph)) # 1.709
    print(get_adjacency_matrix(input_graph))
    


main()

11
7
0.19047619047619047
12.750000000000002
1.709090909090909
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.]
 [0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
